In [1]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter  
import math 
from scipy import stats 

In [2]:
stocks = pd.read_csv('nas.csv')
from secrets import IEX_CLOUD_API_TOKEN
stocks

,Ticker
0,AAPL
1,MSFT
2,AMZN
3,FB
4,TSLA
...,...
96,VRSN
97,DOCU
98,SWKS
99,SGEN


In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'companyName': 'Apple Inc', 'marketcap': 2658238979726, 'week52high': 187.24, 'week52low': 118.36, 'week52highSplitAdjustOnly': 184.59, 'week52lowSplitAdjustOnly': 121.37, 'week52change': 0.12397480618820152, 'sharesOutstanding': 17062210587, 'float': 0, 'avg10Volume': 116012598, 'avg30Volume': 96539830, 'day200MovingAvg': 156.85, 'day50MovingAvg': 180.97, 'employees': 153377, 'ttmEPS': 11.74, 'ttmDividendRate': 0.8819881687181507, 'dividendYield': 0.005422189716348798, 'nextDividendDate': '', 'exDividendDate': '2021-11-05', 'nextEarningsDate': '', 'peRatio': 14.3108907580766, 'beta': 1.33528912487702, 'maxChangePercent': 63.20591712790914, 'year5ChangePercent': 4.781904890929568, 'year2ChangePercent': 1.038881790722329, 'year1ChangePercent': 0.12817893755402027, 'ytdChangePercent': -0.1041566732542958, 'month6ChangePercent': 0.0794985349473861, 'month3ChangePercent': 0.07759899120905037, 'month1ChangePercent': -0.09760000384423385, 'day30ChangePercent': -0.09782976114195707, 'day5Cha

In [4]:
data['year1ChangePercent']

0.12817893755402027

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,AAPL,164.53,0.125032,N/A
1,MSFT,308.01,0.277700,N/A
2,AMZN,2863.29,-0.152236,N/A
3,FB,299.25,0.082645,N/A
4,TSLA,958.10,0.043597,N/A
...,...,...,...,...
96,VRSN,217.64,0.113017,N/A
97,DOCU,115.37,-0.526651,N/A
98,SWKS,143.26,-0.151118,N/A
99,SGEN,128.62,-0.305938,N/A


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe.head()

C:\backup\envs\dev\envs\dev\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MCHP,77.92,0.979498,N/A
1,FTNT,277.77,0.810225,N/A
2,NVDA,231.85,0.667691,N/A
3,ORLY,639.24,0.452697,N/A
4,ODFL,304.04,0.442870,N/A


In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio: 15000


15000


In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MCHP,77.92,0.979498,3
1,FTNT,277.77,0.810225,1
2,NVDA,231.85,0.667691,1
3,ORLY,639.24,0.452697,0
4,ODFL,304.04,0.442870,0
5,MRVL,69.32,0.432884,4
6,LCID,34.53,0.430228,8
7,PAYX,114.20,0.412404,2
8,INTU,514.12,0.390814,0
9,ADP,205.88,0.359274,1


In [11]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [12]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0      0.643564
1      0.831683
2       0.19802
3      0.554455
4      0.485149
         ...   
96     0.613861
97     0.029703
98     0.217822
99      0.09901
100    0.089109
Name: One-Year Return Percentile, Length: 101, dtype: object
0      0.782178
1      0.594059
2      0.178218
3      0.257426
4      0.980198
         ...   
96     0.465347
97     0.009901
98     0.138614
99     0.287129
100    0.267327
Name: Six-Month Return Percentile, Length: 101, dtype: object
0      0.881188
1      0.564356
2      0.306931
3       0.49505
4      0.435644
         ...   
96     0.633663
97     0.019802
98     0.326733
99     0.128713
100    0.118812
Name: Three-Month Return Percentile, Length: 101, dtype: object
0      0.633663
1      0.415842
2      0.277228
3      0.584158
4      0.425743
         ...   
96     0.376238
97     0.118812
98     0.653465
99     0.188119
100    0.841584
Name: One-Month Return Percentile, Length: 101, dtype: object


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,162.15,N/A,0.126547,0.643564,0.081807,0.782178,0.078891,0.881188,-0.096344,0.633663,N/A
1,MSFT,297.73,N/A,0.273858,0.831683,-0.000355,0.594059,-0.062078,0.564356,-0.143796,0.415842,N/A
2,AMZN,2911.24,N/A,-0.156101,0.19802,-0.245785,0.178218,-0.163136,0.306931,-0.181843,0.277228,N/A
3,FB,301.29,N/A,0.080870,0.554455,-0.194390,0.257426,-0.088673,0.49505,-0.105872,0.584158,N/A
4,TSLA,974.61,N/A,0.043710,0.485149,0.440515,0.980198,-0.108821,0.435644,-0.143179,0.425743,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
96,VRSN,209.86,N/A,0.110107,0.613861,-0.044378,0.465347,-0.031933,0.633663,-0.151297,0.376238,N/A
97,DOCU,116.83,N/A,-0.531687,0.029703,-0.626970,0.009901,-0.588233,0.019802,-0.250029,0.118812,N/A
98,SWKS,148.55,N/A,-0.146019,0.217822,-0.277658,0.138614,-0.152650,0.326733,-0.090593,0.653465,N/A
99,SGEN,127.69,N/A,-0.314485,0.09901,-0.143689,0.287129,-0.294715,0.128713,-0.220693,0.188119,N/A


In [13]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [14]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:51]

In [15]:
portfolio_input()


Enter the value of your portfolio: 15000


In [16]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

C:\backup\envs\dev\envs\dev\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,AAPL,162.15,1,0.126547,0.643564,0.081807,0.782178,0.078891,0.881188,-0.096344,0.633663,0.735149
1,MSFT,297.73,0,0.273858,0.831683,-0.000355,0.594059,-0.062078,0.564356,-0.143796,0.415842,0.601485
2,AMZN,2911.24,0,-0.156101,0.19802,-0.245785,0.178218,-0.163136,0.306931,-0.181843,0.277228,0.240099
3,FB,301.29,0,0.080870,0.554455,-0.194390,0.257426,-0.088673,0.49505,-0.105872,0.584158,0.472772
4,TSLA,974.61,0,0.043710,0.485149,0.440515,0.980198,-0.108821,0.435644,-0.143179,0.425743,0.581683
5,GOOG,2646.90,0,0.343865,0.90099,-0.083060,0.39604,-0.087276,0.50495,-0.143010,0.435644,0.559406
6,NVDA,229.46,1,0.655317,0.980198,0.143114,0.871287,-0.037431,0.60396,-0.257217,0.09901,0.638614
7,GOOGL,2690.44,0,0.341520,0.891089,-0.047239,0.455446,-0.077514,0.534653,-0.136388,0.465347,0.586634
8,ADBE,524.90,0,0.063202,0.544554,-0.197265,0.247525,-0.230743,0.227723,-0.117882,0.554455,0.393564
9,PEP,170.40,1,0.258940,0.821782,0.108657,0.80198,0.087234,0.910891,0.009316,0.871287,0.851485


In [17]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [19]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [20]:
writer.save()